In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, Sequential, Input, layers, optimizers, regularizers, callbacks, losses, metrics 
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
# FOR KAGGLE
train_images_dir = '../input/petfinder-pawpularity-score/train/'
test_images_dir = '../input/petfinder-pawpularity-score/test/'

# FOR LOCAL PC
# train_images_dir = './dataset/train/'
# test_images_dir = './dataset/test/'

In [4]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.loc[:, 'filename'] = train_images_dir + train_df['Id'] + '.jpg'

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.loc[:, 'filename'] = test_images_dir + test_df['Id'] + '.jpg'

In [5]:
train_df.head(5)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,filename
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...


In [6]:
img_size = (320, 320, 3)
BATCH_SIZE = 32

def augment_image(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0.95, 1.05)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, 0.95, 1.05)
    image = tf.image.random_hue(image, 0.05)
    return image, label

def parse_image(filename, label=None):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=img_size[2])
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, img_size[:2])
    return image, label

def get_dataset(filenames, label=None, shuffle=False, augment=False, repeat=False):
    dataset = tf.data.TFRecordDataset.from_tensor_slices((filenames, label))     
    dataset = dataset.map(parse_image, num_parallel_calls=4)
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=4)
    if repeat:
        dataset = dataset.repeat()
    if shuffle:
        dataset = dataset.shuffle(1024, reshuffle_each_iteration = True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(True)
    return dataset

In [7]:
def build_model():
    model = Sequential([
        layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),      
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),      
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Flatten(),
        layers.Dense(258, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1)
    ])
    model.compile(optimizer=optimizers.Adam(), 
                  loss=losses.MeanSquaredError(), 
                  metrics=[metrics.RootMeanSquaredError()]
                 )
    return model

es = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.002, patience=10)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=0.000001)

In [8]:
model = build_model()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, val_index in kfold.split(train_df.index, train_df['Pawpularity']):
    train_dataset = get_dataset(train_df.loc[train_index, 'filename'], train_df.loc[train_index, 'Pawpularity'], augment=True)
    val_dataset = get_dataset(train_df.loc[val_index, 'filename'], train_df.loc[val_index, 'Pawpularity'])
    model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    steps_per_epoch=10,
#     use_multiprocessing=True,
#     callbacks=[es, reduce_lr]
)

2021-12-20 01:46:56.496266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 01:46:56.757813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 01:46:56.758562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 01:46:56.759746: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/20


2021-12-20 01:47:01.814211: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


10/10 [==============================] - 26s 2s/step - loss: 1555.4706 - root_mean_squared_error: 39.4395 - val_loss: 1835.4403 - val_root_mean_squared_error: 42.8420
Epoch 2/20
10/10 [==============================] - 15s 2s/step - loss: 612.1815 - root_mean_squared_error: 24.7423 - val_loss: 1696.3433 - val_root_mean_squared_error: 41.1867
Epoch 3/20
10/10 [==============================] - 15s 2s/step - loss: 654.2552 - root_mean_squared_error: 25.5784 - val_loss: 1606.4456 - val_root_mean_squared_error: 40.0805
Epoch 4/20
10/10 [==============================] - 15s 2s/step - loss: 587.1573 - root_mean_squared_error: 24.2313 - val_loss: 1425.6569 - val_root_mean_squared_error: 37.7579
Epoch 5/20
10/10 [==============================] - 15s 2s/step - loss: 551.6627 - root_mean_squared_error: 23.4875 - val_loss: 1402.0005 - val_root_mean_squared_error: 37.4433
Epoch 6/20
10/10 [==============================] - 15s 2s/step - loss: 592.6501 - root_mean_squared_error: 24.3444 - val_los

In [9]:
test_dataset = get_dataset(test_df['filename'])

In [10]:
preds = model.predict(test_dataset)

In [11]:
submisson_df = test_df[['Id']]
submisson_df['Pawpularity'] = preds

In [12]:
submisson_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,30.946404
1,43a2262d7738e3d420d453815151079e,30.946795
2,4e429cead1848a298432a0acad014c9d,30.956884
3,80bc3ccafcc51b66303c2c263aa38486,30.938499
4,8f49844c382931444e68dffbe20228f4,30.951050
5,b03f7041962238a7c9d6537e22f9b017,30.955084
6,c978013571258ed6d4637f6e8cc9d6a3,30.950874
7,e0de453c1bffc20c22b072b34b54e50f,30.954060


In [13]:
submisson_df.to_csv('submission.csv', index=False)

In [14]:
model.save('/kaggle/working/cnn_images_only_model.h5')